In [83]:
import nltk
from nltk.util import ngrams
from collections import Counter
import string
import numpy as np
import pandas as pd

In [ ]:
nltk.download('punkt')

In [8]:
from google.colab import files
uploaded = files.upload()

Saving IMDB Dataset.csv to IMDB Dataset.csv


In [14]:
df = pd.read_csv('IMDB Dataset.csv')

In [72]:
print(df.head())


task1file = df

print(task1file.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [73]:
print(task1file.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [74]:
def preprocess_review(text):
    # Remove <br /> tags
    text = text.replace('<br />', ' ')
    # Replace multiple periods with a single space
    text = text.replace('..', ' ').replace('...', ' ')
    # Replace periods with a space and remove surrounding spaces
    text = text.replace('.', ' ').replace(',', ' ')
    # Replace multiple spaces with a single space
    text = ' '.join(text.split())
    return text.lower()


task1file['review'] = task1file['review'].apply(preprocess_review)
firstele = task1file.iloc[0]['review']
print(firstele)


one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked they are right as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to many aryans muslims gangstas latinos christians italians irish and more so scuffles death stares dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare forget pretty 

In [75]:

#tokenization for unigrram model

def tokenize_unigram(text):
    return text.split()  # Split by whitespace to get tokens

# Create a list to store all tokens
all_tokens_unigrams = []

# Tokenize each review and collect tokens
for review in task1file['review']:
    tokens_uni = tokenize_unigram(review)
    all_tokens_unigrams.extend(tokens_uni)

token_counts_uni = Counter(all_tokens_unigrams)

# Calculate total number of tokens
total_tokens = sum(token_counts_uni.values())

# Calculate probabilities for each token
token_probabilities = {token: count / total_tokens for token, count in token_counts_uni.items()}

# Display results
N = 10  # Number of top tokens to display

# Get the top N token counts
print(f"Top {N} Token Counts:")
for token, count in token_counts_uni.most_common(N):
    print(f"{token}: {count}")

# Get the top N token probabilities
print(f"\nTop {N} Token Probabilities:")
for token in list(token_counts_uni.keys())[:N]:  # First N tokens in the same order
    print(f"{token}: {token_probabilities[token]:.4f}")

Top 10 Token Counts:
the: 655908
a: 319358
and: 319242
of: 287963
to: 266551
is: 209872
in: 183939
it: 152384
i: 151559
this: 148933

Top 10 Token Probabilities:
one: 0.0044
of: 0.0251
the: 0.0571
other: 0.0016
reviewers: 0.0000
has: 0.0029
mentioned: 0.0001
that: 0.0118
after: 0.0013
watching: 0.0008


In [76]:
def get_most_used_word(token_counts_uni):
    # Get the most common word and its count
    most_common_word, most_common_count = token_counts_uni.most_common(1)[0]  # Get the first element
    return most_common_word, most_common_count



In [77]:
# Generate bigrams
all_bigrams = [(all_tokens_unigrams[i], all_tokens_unigrams[i + 1]) for i in range(len(all_tokens_unigrams) - 1)]

# Count frequencies of each bigram
bigram_counts = Counter(all_bigrams)

# Calculate total number of bigrams
total_bigrams = sum(bigram_counts.values())

# Calculate probabilities for each bigram
bigram_probabilities = {bigram: count / total_bigrams for bigram, count in bigram_counts.items()}

# Display results
N = 10  # Number of top bigrams to display

# Get the top N bigram counts
print(f"Top {N} Bigram Counts:")
for bigram, count in bigram_counts.most_common(N):
    print(f"{bigram}: {count}")

# Get the top N bigram probabilities
print(f"\nTop {N} Bigram Probabilities:")
for bigram in list(bigram_counts.keys())[:N]:  # First N bigrams in the same order
    print(f"{bigram}: {bigram_probabilities[bigram]:.4f}")

Top 10 Bigram Counts:
('of', 'the'): 76422
('in', 'the'): 49376
('this', 'movie'): 30301
('and', 'the'): 26017
('is', 'a'): 25959
('the', 'film'): 24358
('to', 'the'): 23511
('to', 'be'): 23148
('the', 'movie'): 22574
('this', 'film'): 20841

Top 10 Bigram Probabilities:
('one', 'of'): 0.0014
('of', 'the'): 0.0067
('the', 'other'): 0.0004
('other', 'reviewers'): 0.0000
('reviewers', 'has'): 0.0000
('has', 'mentioned'): 0.0000
('mentioned', 'that'): 0.0000
('that', 'after'): 0.0000
('after', 'watching'): 0.0001
('watching', 'just'): 0.0000


In [78]:
def predict_next_word(bigram_counts, last_word):
    # Find all bigrams that start with the last_word
    possible_bigrams = {bigram: count for bigram, count in bigram_counts.items() if bigram[0] == last_word}

    # If there are no possible bigrams, return a message
    if not possible_bigrams:
        return "No predictions available."

    # Find the bigram with the highest count
    predicted_bigram = max(possible_bigrams, key=possible_bigrams.get)

    # Return the next word in the bigram
    return predicted_bigram[1]

# Example usage
last_input_word = "this"  # Replace this with the last word from your input sentence
predicted_word = predict_next_word(bigram_counts, last_input_word)

print(f"Next word after '{last_input_word}': {predicted_word}")

Next word after 'this': movie


In [79]:
all_trigrams = [(all_tokens_unigrams[i], all_tokens_unigrams[i + 1], all_tokens_unigrams[i + 2])
                for i in range(len(all_tokens_unigrams) - 2)]

# Count frequencies of each trigram
trigram_counts = Counter(all_trigrams)

# Display the first 10 trigram counts
print("First 10 Trigram Counts:")
for trigram, count in trigram_counts.most_common(10):
    print(f"{trigram}: {count}")

First 10 Trigram Counts:
('one', 'of', 'the'): 9716
('this', 'movie', 'is'): 5196
('this', 'is', 'a'): 4764
('a', 'lot', 'of'): 4678
('of', 'the', 'film'): 4677
('of', 'the', 'movie'): 4069
('some', 'of', 'the'): 3754
('the', 'film', 'is'): 3650
('is', 'one', 'of'): 3542
('this', 'film', 'is'): 3457


In [69]:
def predict_next_word(last_two_words):
    # Split the input to get the last two words

    words = last_two_words.lower()
    words = words.split()

    if len(words) != 2:
        print("Please provide exactly two words.")
        return None

    # Create a prefix from the last two words
    prefix = tuple(words)

    # Find all trigrams that start with the prefix
    possible_trigrams = [(trigram, count) for trigram, count in trigram_counts.items() if trigram[:2] == prefix]

    if not possible_trigrams:
        print("No predictions available for the provided words.")
        return None

    # Count occurrences of the third words
    third_word_counts = Counter(trigram[2] for trigram, count in possible_trigrams)

    # Get the most common third word
    most_common_third_word = third_word_counts.most_common(1)[0][0]

    return most_common_third_word

# Example usage
last_two = "Derrick Cannon"  # Change this to test with different words
predicted_word = predict_next_word(last_two)
print(f"Predicted next word: {predicted_word}")

Predicted next word: and


                                              review sentiment
0  one of the other reviewers has mentioned that ...  positive
1  a wonderful little production the filming tech...  positive
2  i thought this was a wonderful way to spend ti...  positive
3  basically there's a family where a little boy ...  negative
4  petter mattei's "love in the time of money" is...  positive


In [89]:
# Second Question



train_data = task1file.iloc[:45000]  # First 45,000 rows for training
test_data = task1file.iloc[45000:]    # Remaining 5,000 rows for testing

reviews_train = train_data['review'].tolist()
sentiments_train = train_data['sentiment'].tolist()
reviews_test = test_data['review'].tolist()
sentiments_test = test_data['sentiment'].tolist()


# Assuming 'reviews' is your preprocessed review text and 'labels' contains sentiment
all_words = ' '.join(reviews_train).split()
vocabulary = set(all_words)
vocabulary_size = len(vocabulary)



label_counts = Counter(sentiments_train)
total_reviews = len(sentiments_train)

prior_probabilities = {label: count / total_reviews for label, count in label_counts.items()}

# Initialize counts for each class
likelihoods = {label: Counter() for label in label_counts.keys()}

for review, label in zip(reviews_train, sentiments_train):
    words = review.split()
    likelihoods[label].update(words)

# Calculate conditional probabilities with Laplace smoothing
conditional_probabilities = {}
for label, counts in likelihoods.items():
    total_words_in_class = sum(counts.values())
    conditional_probabilities[label] = {word: (count + 1) / (total_words_in_class + vocabulary_size)
                                        for word, count in counts.items()}







In [90]:
def predict_review_sentiment(review):
    words = review.split()
    log_probs = {label: 0 for label in prior_probabilities.keys()}

    for label in prior_probabilities.keys():
        log_probs[label] = np.log(prior_probabilities[label])  # Start with log of prior

        for word in words:
            if word in conditional_probabilities[label]:
                log_probs[label] += np.log(conditional_probabilities[label][word])
            else:
                # Use the smoothed probability for unseen words
                log_probs[label] += np.log(1 / (sum(likelihoods[label].values()) + vocabulary_size))

    # Choose the class with the highest log probability
    return max(log_probs, key=log_probs.get)






In [97]:
correct_predictions = 0
#for unigram model
for review, label in zip(reviews_test, sentiments_test):
    predicted_label = predict_review_sentiment(review)
    if predicted_label == label:
        correct_predictions += 1

from sklearn.metrics import classification_report

print(classification_report(test_data['sentiment'], test_data['predicted_sentiment']))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00         8
    positive       1.00      1.00      1.00         2

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10



In [95]:
from collections import defaultdict, Counter


# Load your dataset (assumed to be a CSV file with 'review' and 'sentiment' columns)
# task1file = pd.read_csv('your_file.csv')

# Assume task1file is already preprocessed
# Split the data into training and test sets
train_size = 49990
train_data = task1file[:train_size]
test_data = task1file[train_size:]

# Create bigrams
def create_bigrams(text):
    words = text.split()
    return [(words[i], words[i + 1]) for i in range(len(words) - 1)]

# Calculate prior and conditional probabilities
def calculate_probabilities(data):
    total_reviews = len(data)
    positive_reviews = data[data['sentiment'] == 'positive']
    negative_reviews = data[data['sentiment'] == 'negative']

    prior_positive = len(positive_reviews) / total_reviews
    prior_negative = len(negative_reviews) / total_reviews

    # Bigram counts
    bigram_counts_positive = Counter()
    bigram_counts_negative = Counter()

    # Populate the bigram counts
    for review in positive_reviews['review']:
        bigrams = create_bigrams(review)
        bigram_counts_positive.update(bigrams)

    for review in negative_reviews['review']:
        bigrams = create_bigrams(review)
        bigram_counts_negative.update(bigrams)

    # Total bigrams in each class
    total_bigrams_positive = sum(bigram_counts_positive.values())
    total_bigrams_negative = sum(bigram_counts_negative.values())

    # Conditional probabilities with Laplace smoothing
    bigram_prob_positive = {bigram: (count + 1) / (total_bigrams_positive + len(bigram_counts_positive))
                            for bigram, count in bigram_counts_positive.items()}
    bigram_prob_negative = {bigram: (count + 1) / (total_bigrams_negative + len(bigram_counts_negative))
                            for bigram, count in bigram_counts_negative.items()}

    return prior_positive, prior_negative, bigram_prob_positive, bigram_prob_negative

# Function to predict sentiment of a review
def predict_sentiment(review, prior_positive, prior_negative, bigram_prob_positive, bigram_prob_negative):
    bigrams = create_bigrams(review)
    log_prob_positive = np.log(prior_positive)
    log_prob_negative = np.log(prior_negative)

    # Calculate the log probability for each class
    for bigram in bigrams:
        log_prob_positive += np.log(bigram_prob_positive.get(bigram, 1 / (sum(bigram_prob_positive.values()) + len(bigram_prob_positive))))
        log_prob_negative += np.log(bigram_prob_negative.get(bigram, 1 / (sum(bigram_prob_negative.values()) + len(bigram_prob_negative))))

    return 'positive' if log_prob_positive > log_prob_negative else 'negative'

# Calculate probabilities from training data
prior_positive, prior_negative, bigram_prob_positive, bigram_prob_negative = calculate_probabilities(train_data)

# Test the model
test_data['predicted_sentiment'] = test_data['review'].apply(
    lambda x: predict_sentiment(x, prior_positive, prior_negative, bigram_prob_positive, bigram_prob_negative)
)

# Evaluate the model
from sklearn.metrics import classification_report

print(classification_report(test_data['sentiment'], test_data['predicted_sentiment']))


              precision    recall  f1-score   support

    negative       1.00      0.75      0.86         8
    positive       0.50      1.00      0.67         2

    accuracy                           0.80        10
   macro avg       0.75      0.88      0.76        10
weighted avg       0.90      0.80      0.82        10



<ipython-input-95-391a5a035f8a>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predicted_sentiment'] = test_data['review'].apply(


In [96]:
from collections import defaultdict, Counter
import numpy as np
import pandas as pd

# Load your dataset (assumed to be a CSV file with 'review' and 'sentiment' columns)
# task1file = pd.read_csv('your_file.csv')

# Assume task1file is already preprocessed
# Split the data into training and test sets
train_size = 49990
train_data = task1file[:train_size]
test_data = task1file[train_size:]

# Create trigrams
def create_trigrams(text):
    words = text.split()
    return [(words[i], words[i + 1], words[i + 2]) for i in range(len(words) - 2)]

# Calculate prior and conditional probabilities
def calculate_probabilities(data):
    total_reviews = len(data)
    positive_reviews = data[data['sentiment'] == 'positive']
    negative_reviews = data[data['sentiment'] == 'negative']

    prior_positive = len(positive_reviews) / total_reviews
    prior_negative = len(negative_reviews) / total_reviews

    # Trigram counts
    trigram_counts_positive = Counter()
    trigram_counts_negative = Counter()

    # Populate the trigram counts
    for review in positive_reviews['review']:
        trigrams = create_trigrams(review)
        trigram_counts_positive.update(trigrams)

    for review in negative_reviews['review']:
        trigrams = create_trigrams(review)
        trigram_counts_negative.update(trigrams)

    # Total trigrams in each class
    total_trigrams_positive = sum(trigram_counts_positive.values())
    total_trigrams_negative = sum(trigram_counts_negative.values())

    # Conditional probabilities with Laplace smoothing
    trigram_prob_positive = {trigram: (count + 1) / (total_trigrams_positive + len(trigram_counts_positive))
                             for trigram, count in trigram_counts_positive.items()}
    trigram_prob_negative = {trigram: (count + 1) / (total_trigrams_negative + len(trigram_counts_negative))
                             for trigram, count in trigram_counts_negative.items()}

    return prior_positive, prior_negative, trigram_prob_positive, trigram_prob_negative

# Function to predict sentiment of a review
def predict_sentiment(review, prior_positive, prior_negative, trigram_prob_positive, trigram_prob_negative):
    trigrams = create_trigrams(review)
    log_prob_positive = np.log(prior_positive)
    log_prob_negative = np.log(prior_negative)

    # Calculate the log probability for each class
    for trigram in trigrams:
        log_prob_positive += np.log(trigram_prob_positive.get(trigram, 1 / (sum(trigram_prob_positive.values()) + len(trigram_prob_positive))))
        log_prob_negative += np.log(trigram_prob_negative.get(trigram, 1 / (sum(trigram_prob_negative.values()) + len(trigram_prob_negative))))

    return 'positive' if log_prob_positive > log_prob_negative else 'negative'

# Calculate probabilities from training data
prior_positive, prior_negative, trigram_prob_positive, trigram_prob_negative = calculate_probabilities(train_data)

# Test the model
test_data['predicted_sentiment'] = test_data['review'].apply(
    lambda x: predict_sentiment(x, prior_positive, prior_negative, trigram_prob_positive, trigram_prob_negative)
)

# Evaluate the model
from sklearn.metrics import classification_report

print(classification_report(test_data['sentiment'], test_data['predicted_sentiment']))


              precision    recall  f1-score   support

    negative       1.00      1.00      1.00         8
    positive       1.00      1.00      1.00         2

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10



<ipython-input-96-9209e3e524d9>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predicted_sentiment'] = test_data['review'].apply(


All the methods were applied in this assignment. Different approaches were used for the first and the seconde part. The firs part was simple as i caluclated the prbablities of group of words. One for unigram 2 and three for trigram. They were tokenized inn that manner. When getting one word it used the probablities to predict the next word. for unigram the most repeated word was given. For bi and trgrams it looked up the dictionary to see what the most popularr combination was.


For the second part different models were trained using the bayseian calculation. For bigrams and trigrams the tokens had their probablity assigned in realtion to whether they appeared in psitiive or negative reviews. When the test set is given it checks where each unigram bigram or trigaram sequence is present in which sentiment the most and calculates the logs to determine the output.